In [ ]:
import requests
import pandas as pd
import time

TOKEN = "129ac2e3-e8a6-72c7-58c1-acced5a601bd"

indicadores_pib = {
    "746097": "Total Nacional",
    "746196": "Primarias",
    "746229": "Agricultura, cría y explotación de animales, aprovechamiento forestal, pesca y caza",
    "746262": "Agricultura",
    "746295": "Cría y explotación de animales",
    "746328": "Pesca, caza y captura",
    "746361": "Aprovechamiento forestal",
    "746394": "Secundarias",
    "746427": "Minería",
    "746460": "Minería petrolera",
    "746493": "Minería no petrolera",
    "746526": "Generación, transmisión, distribución y comercialización de energía eléctrica, suministro de agua y de gas",
    "746559": "Construcción",
    "746592": "Manufacturas",
    "746625": "Alimentarias",
    "746658": "Bebidas y tabaco",
    "746691": "Insumos, acabados y productos textiles",
    "746724": "Prendas de vestir y productos de cuero y piel",
    "746757": "Madera",
    "746790": "Papel",
    "746823": "Productos derivados del petróleo y carbón, química, plástico y hule",
    "746856": "Productos a base de minerales no metálicos",
    "746889": "Metálicas básicas y productos metálicos",
    "746922": "Maquinaria y equipo, computación, electrónicos y accesorios",
    "746955": "Muebles, colchones y persianas",
    "746988": "Otras industrias manufactureras",
    "747021": "Terciarias",
    "747054": "Comercio al por mayor",
    "747087": "Comercio al por menor",
    "747120": "Transportes, correos y almacenamiento",
    "747153": "Información en medio masivos",
    "747186": "Servicios financieros y de seguros",
    "747219": "Servicios inmobiliarios y de alquiler de bienes muebles e intangibles",
    "747252": "Servicios profesionales, científicos y técnicos",
    "747285": "Corporativos",
    "747318": "Servicios de apoyo a los negocios",
    "747351": "Servicios educativos",
    "747384": "Servicios de salud y asistencia social",
    "747417": "Servicios de esparcimiento",
    "747450": "Servicios de alojamiento temporal y preparación de alimentos y bebidas",
    "747483": "Otros servicios excepto actividades gubernamentales",
    "747516": "Actividades gubernamentales"
}

resultados_totales = []
consultas_fallidas = []

def extraer_datos(ind_clave, ind_nombre, clave_estado, url):
    for intento in range(3):
        try:
            response = requests.get(url, timeout=15)
            
            if response.status_code == 200:
                data = response.json()
                if 'Series' in data and len(data['Series']) > 0:
                    serie = data['Series'][0]
                    if 'OBSERVATIONS' in serie:
                        for obs in serie['OBSERVATIONS']:
                            registro = {
                                'CLAVE_INDICADOR': ind_clave,
                                'SECTOR': ind_nombre,
                                'CVE_ENT': obs.get('COBER_GEO', clave_estado),
                                'TIME_PERIOD': obs.get('TIME_PERIOD'),
                                'OBS_VALUE': obs.get('OBS_VALUE')
                            }
                            resultados_totales.append(registro)
                return True
            else:
                print(f"  ⚠️ Error HTTP {response.status_code} (Intento {intento+1}/3)")
                time.sleep(2)
                
        except Exception as e:
            print(f"  ⚠️ Fallo de red: {type(e).__name__} (Intento {intento+1}/3)")
            time.sleep(2)
            
    return False

print("--- INICIANDO EXTRACCIÓN HISTÓRICA MASIVA DEL PIB ---")

for ind_clave, ind_nombre in indicadores_pib.items():
    print(f"Consultando: {ind_clave} - {ind_nombre}")
    
    for i in range(0, 33):
        clave_estado = f"{i:02d}"
        url = f"https://www.inegi.org.mx/app/api/indicadores/desarrolladores/jsonxml/INDICATOR/{ind_clave}/es/{clave_estado}/false/BIE-BISE/2.0/{TOKEN}?type=json"
        
        exito = extraer_datos(ind_clave, ind_nombre, clave_estado, url)
        
        if not exito:
            print(f"  ❌ Fallo total en {ind_clave}-{clave_estado}. Se envía a la cola final.")
            consultas_fallidas.append((ind_clave, ind_nombre, clave_estado, url))
            
        time.sleep(0.3)

if consultas_fallidas:
    print("\n--- INICIANDO REINTENTOS DE CONSULTAS FALLIDAS ---")
    for ind_clave, ind_nombre, clave_estado, url in consultas_fallidas:
        print(f"Reintentando: {ind_clave} - Estado {clave_estado}")
        extraer_datos(ind_clave, ind_nombre, clave_estado, url)
        time.sleep(0.5)

if resultados_totales:
    df_pib = pd.DataFrame(resultados_totales)
    
    df_pib['OBS_VALUE'] = pd.to_numeric(df_pib['OBS_VALUE'], errors='coerce')
    df_pib['TIME_PERIOD'] = pd.to_numeric(df_pib['TIME_PERIOD'], errors='coerce')
    
    df_pib = df_pib.sort_values(by=['CLAVE_INDICADOR', 'CVE_ENT', 'TIME_PERIOD'])
    
    df_final = df_pib.groupby(['CLAVE_INDICADOR', 'CVE_ENT']).tail(2).reset_index(drop=True)
    
    print("\n--- RESUMEN FINAL ---")
    print(f"Total de registros procesados y filtrados: {len(df_final)}")
    print(df_final.head(10))
else:
    print("\nNo se encontraron datos.")

--- INICIANDO EXTRACCIÓN HISTÓRICA MASIVA DEL PIB ---
Consultando: 746097 - Total Nacional
Consultando: 746196 - Primarias
Consultando: 746229 - Agricultura, cría y explotación de animales, aprovechamiento forestal, pesca y caza
Consultando: 746262 - Agricultura
Consultando: 746295 - Cría y explotación de animales
  ⚠️ Fallo de red: ConnectionError (Intento 1/3)
  ⚠️ Fallo de red: ConnectionError (Intento 2/3)
Consultando: 746328 - Pesca, caza y captura
Consultando: 746361 - Aprovechamiento forestal
Consultando: 746394 - Secundarias
  ⚠️ Fallo de red: ReadTimeout (Intento 1/3)
  ⚠️ Fallo de red: ConnectTimeout (Intento 1/3)
Consultando: 746427 - Minería
  ⚠️ Fallo de red: ConnectionError (Intento 1/3)
Consultando: 746460 - Minería petrolera
Consultando: 746493 - Minería no petrolera
  ⚠️ Fallo de red: ConnectionError (Intento 1/3)
Consultando: 746526 - Generación, transmisión, distribución y comercialización de energía eléctrica, suministro de agua y de gas
  ⚠️ Fallo de red: Connectio